<a href="https://colab.research.google.com/github/mheronanh/MBKM-Xirka-HRM/blob/main/main/Software/CNN-based/Training%20VGG16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split
import os.path

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten

from tensorflow import keras

from sklearn.metrics import r2_score

In [ ]:
image_dir = Path('/content/drive/MyDrive/Data/CNN PCA/source2')

In [ ]:
filepaths = pd.Series(list(image_dir.glob(r'**/*.png')), name='Filepath').astype(str)

In [ ]:
bpms = pd.Series(filepaths.apply(lambda x: (os.path.split(os.path.split(x)[1])[1])[:-4]), name='BPMS').astype(np.float64)

In [ ]:
images = pd.concat([filepaths, bpms], axis=1).sample(frac=1.0, random_state=1).reset_index(drop=True)

In [ ]:
image_df = images.sample(len(images), random_state = 1).reset_index(drop=True)
train_df, test_df = train_test_split(image_df, train_size= 0.7, shuffle= True, random_state=1)

In [ ]:
len(images)

739

In [ ]:
train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    validation_split=0.4
)

test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
)

train_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='BPMS',
    target_size=(216, 144),
    color_mode='rgb',
    class_mode='raw',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='training'
)

val_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='BPMS',
    target_size=(216, 144),
    color_mode='rgb',
    class_mode='raw',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='validation'
)

test_images = test_generator.flow_from_dataframe(
    dataframe=test_df,
    x_col='Filepath',
    y_col='BPMS',
    target_size=(216, 144),
    color_mode='rgb',
    class_mode='raw',
    batch_size=32,
    shuffle=False
)

Found 311 validated image filenames.
Found 206 validated image filenames.
Found 222 validated image filenames.


In [ ]:
base_model = keras.applications.VGG16(
    weights='imagenet',  # Load weights pre-trained on ImageNet.
    input_shape=(216, 144, 3),
    include_top=False)  # Do not include the ImageNet classifier at the top.

58900480/58889256 [==============================] - 0s 0us/step


In [ ]:
base_model.trainable = False

In [ ]:
inputs = keras.Input(shape=(216, 144, 3))
# We make sure that the base_model is running in inference mode here,
# by passing `training=False`. This is important for fine-tuning, as you will
# learn in a few paragraphs.
x = base_model(inputs, training=False)
# Convert features of shape `base_model.output_shape[1:]` to vectors
x = keras.layers.GlobalAveragePooling2D()(x)
# A Dense classifier with a single unit (binary classification)
outputs = keras.layers.Dense(1, activation='linear')(x)
model = keras.Model(inputs, outputs)

In [ ]:
model.compile(
    optimizer='adam',
    loss='mae'
)

history = model.fit(
    train_images,
    validation_data=val_images,
    epochs=50,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=5,
            restore_best_weights=True
        )
    ]
)

Epoch 1/50
10/10 [==============================] - 302s 30s/step - loss: 87.9104 - val_loss: 82.9524
Epoch 2/50
10/10 [==============================] - 161s 17s/step - loss: 79.8305 - val_loss: 74.9459
Epoch 3/50
10/10 [==============================] - 179s 19s/step - loss: 71.9295 - val_loss: 66.9417
Epoch 4/50
10/10 [==============================] - 161s 17s/step - loss: 63.8760 - val_loss: 58.9446
Epoch 5/50
10/10 [==============================] - 163s 17s/step - loss: 55.8203 - val_loss: 50.9428
Epoch 6/50
10/10 [==============================] - 162s 17s/step - loss: 47.8946 - val_loss: 42.9427
Epoch 7/50
10/10 [==============================] - 162s 17s/step - loss: 39.9071 - val_loss: 34.9372
Epoch 8/50
10/10 [==============================] - 164s 17s/step - loss: 32.0676 - val_loss: 26.9610
Epoch 9/50
10/10 [==============================] - 163s 17s/step - loss: 24.4663 - val_loss: 19.9815
Epoch 10/50
10/10 [==============================] - 164s 17s/step - loss: 18.1547

In [ ]:
from keras.models import model_from_json
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


**Training #2**

In [ ]:
from keras.models import model_from_json
# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")

Loaded model from disk


In [ ]:
opt = keras.optimizers.Adam(learning_rate=0.0001)
loaded_model.compile(
    optimizer=opt,
    loss='mae'
)

history = loaded_model.fit(
    train_images,
    validation_data=val_images,
    epochs=20,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=5,
            restore_best_weights=True
        )
    ]
)

Epoch 1/20
10/10 [==============================] - 163s 17s/step - loss: 10.6329 - val_loss: 10.8153
Epoch 2/20
10/10 [==============================] - 162s 17s/step - loss: 10.6253 - val_loss: 10.8138
Epoch 3/20
10/10 [==============================] - 162s 17s/step - loss: 10.6194 - val_loss: 10.8136
Epoch 4/20
10/10 [==============================] - 162s 17s/step - loss: 10.6171 - val_loss: 10.8098
Epoch 5/20
10/10 [==============================] - 162s 17s/step - loss: 10.6115 - val_loss: 10.8078
Epoch 6/20
10/10 [==============================] - 162s 17s/step - loss: 10.6077 - val_loss: 10.8062
Epoch 7/20
10/10 [==============================] - ETA: 0s - loss: 10.6040

KeyboardInterrupt: ignored

In [ ]:
predicted_bpm = np.squeeze(model.predict(test_images))
true_bpm  = test_images.labels

rmse = np.sqrt(model.evaluate(test_images, verbose=0))
print('Test RMSE: {:.5f}'.format(rmse))

r2 = r2_score(true_bpm, predicted_bpm)
print('Test R2 Score: {:.5f}'.format(r2))

Test RMSE: 3.12256
Test R2 Score: -0.06064


In [ ]:
max(predicted_bpm)

88.94751

In [ ]:
from keras.models import model_from_json
# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")

Loaded model from disk


In [ ]:
opt = keras.optimizers.Adam(learning_rate=0.0001)
loaded_model.compile(
    optimizer=opt,
    loss='mae'
)

history = loaded_model.fit(
    train_images,
    validation_data=val_images,
    epochs=20,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=5,
            restore_best_weights=True
        )
    ]
)

Epoch 1/20
22/22 [==============================] - 355s 16s/step - loss: 9.1366 - val_loss: 10.8253
Epoch 2/20
22/22 [==============================] - 359s 17s/step - loss: 9.1184 - val_loss: 10.8258
Epoch 3/20
22/22 [==============================] - 360s 17s/step - loss: 9.1125 - val_loss: 10.8247
Epoch 4/20
22/22 [==============================] - 356s 16s/step - loss: 9.1128 - val_loss: 10.8241
Epoch 5/20
22/22 [==============================] - 356s 16s/step - loss: 9.1120 - val_loss: 10.8217
Epoch 6/20
22/22 [==============================] - 356s 16s/step - loss: 9.1085 - val_loss: 10.8212
Epoch 7/20
22/22 [==============================] - 356s 16s/step - loss: 9.1077 - val_loss: 10.8213
Epoch 8/20
22/22 [==============================] - 355s 16s/step - loss: 9.1039 - val_loss: 10.8205
Epoch 9/20
22/22 [==============================] - 355s 16s/step - loss: 9.1037 - val_loss: 10.8191
Epoch 10/20
22/22 [==============================] - 355s 17s/step - loss: 9.1019 - val_los

In [ ]:
predicted_bpm = np.squeeze(loaded_model.predict(test_images))
true_bpm  = test_images.labels

rmse = np.sqrt(loaded_model.evaluate(test_images, verbose=0))
print('Test RMSE: {:.5f}'.format(rmse))

r2 = r2_score(true_bpm, predicted_bpm)
print('Test R2 Score: {:.5f}'.format(r2))

Test RMSE: 3.12005
Test R2 Score: -0.04595


In [ ]:
np.mean(np.abs(predicted_bpm-true_bpm))

9.734738310185408

In [ ]:
# serialize model to JSON
model_json = loaded_model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
loaded_model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


In [ ]:
from keras.models import model_from_json
# load json and create model
json_file = open('model.json', 'r')
model_json = json_file.read()
json_file.close()
model = model_from_json(model_json)
# load weights into new model
model.load_weights("model.h5")
print("Loaded model from disk")

Loaded model from disk


In [ ]:
opt = keras.optimizers.Adam(learning_rate=0.00001)
loaded_model.compile(
    optimizer=opt,
    loss='mae'
)

history = loaded_model.fit(
    train_images,
    validation_data=val_images,
    epochs=10,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=5,
            restore_best_weights=True
        )
    ]
)

Epoch 1/10
22/22 [==============================] - 356s 16s/step - loss: 9.0843 - val_loss: 10.8126
Epoch 2/10
22/22 [==============================] - 356s 16s/step - loss: 9.0836 - val_loss: 10.8126
Epoch 3/10
22/22 [==============================] - 356s 16s/step - loss: 9.0836 - val_loss: 10.8126
Epoch 4/10
22/22 [==============================] - 356s 16s/step - loss: 9.0829 - val_loss: 10.8124
Epoch 5/10
22/22 [==============================] - 356s 17s/step - loss: 9.0831 - val_loss: 10.8123
Epoch 6/10
22/22 [==============================] - 357s 17s/step - loss: 9.0826 - val_loss: 10.8121
Epoch 7/10
22/22 [==============================] - 357s 17s/step - loss: 9.0825 - val_loss: 10.8120
Epoch 8/10
22/22 [==============================] - 357s 17s/step - loss: 9.0829 - val_loss: 10.8120
Epoch 9/10
22/22 [==============================] - 357s 17s/step - loss: 9.0824 - val_loss: 10.8119
Epoch 10/10
22/22 [==============================] - 356s 16s/step - loss: 9.0822 - val_los

In [ ]:
from keras.models import model_from_json
# load json and create model
json_file = open('model.json', 'r')
model_json = json_file.read()
json_file.close()
model = model_from_json(model_json)
# load weights into new model
model.load_weights("model.h5")
print("Loaded model from disk")

Loaded model from disk


In [ ]:
for i in range(1,10):  
  test_image_dir = Path('/content/drive/MyDrive/Data/CNN ICA/source2/v{}'.format(i))
  test_filepaths = pd.Series(list(test_image_dir.glob(r'**/*.png')), name='Filepath').astype(str)
  test_bpms = pd.Series(test_filepaths.apply(lambda x: (os.path.split(os.path.split(x)[1])[1])[:-4]), name='BPMS').astype(np.float64)
  test_images = pd.concat([test_filepaths, test_bpms], axis=1).sample(frac=1.0, random_state=1).reset_index(drop=True)
  test_image_df = test_images.sample(len(test_images), random_state = 1).reset_index(drop=True)
  test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
  )
  test_images_flow = test_generator.flow_from_dataframe(
      dataframe=test_image_df,
      x_col='Filepath',
      y_col='BPMS',
      target_size=(216, 144),
      color_mode='rgb',
      class_mode='raw',
      batch_size=32,
      shuffle=False
  )
  predicted_bpm = np.squeeze(model.predict(test_images_flow))
  true_bpm  = test_images_flow.labels
  print("v" + str(i) + ": " + str(np.mean(np.abs(predicted_bpm-true_bpm))))

Found 107 validated image filenames.
v1: 8.21073215110922
Found 49 validated image filenames.
v2: 8.672826237697386
Found 103 validated image filenames.
v3: 8.431057295547973
Found 106 validated image filenames.
v4: 7.422072099514161
Found 107 validated image filenames.
v5: 7.070457216765391
Found 106 validated image filenames.
v6: 7.952692159044876
Found 105 validated image filenames.
v7: 21.639760209077576
Found 96 validated image filenames.
v8: 7.224693856064445
Found 104 validated image filenames.
v9: 8.509395839531134


In [ ]:
for i in range(1,3):  
  test_image_dir = Path('/content/drive/MyDrive/Data/CNN ICA/source3/v{}'.format(i))
  test_filepaths = pd.Series(list(test_image_dir.glob(r'**/*.png')), name='Filepath').astype(str)
  test_bpms = pd.Series(test_filepaths.apply(lambda x: (os.path.split(os.path.split(x)[1])[1])[:-4]), name='BPMS').astype(np.float64)
  test_images = pd.concat([test_filepaths, test_bpms], axis=1).sample(frac=1.0, random_state=1).reset_index(drop=True)
  test_image_df = test_images.sample(len(test_images), random_state = 1).reset_index(drop=True)
  test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
  )
  test_images_flow = test_generator.flow_from_dataframe(
      dataframe=test_image_df,
      x_col='Filepath',
      y_col='BPMS',
      target_size=(216, 144),
      color_mode='rgb',
      class_mode='raw',
      batch_size=32,
      shuffle=False
  )
  predicted_bpm = np.squeeze(model.predict(test_images_flow))
  true_bpm  = test_images_flow.labels
  print("v" + str(i) + ": " + str(np.mean(np.abs(predicted_bpm-true_bpm))))

Found 104 validated image filenames.
v1: 8.24624493970874
Found 106 validated image filenames.
v2: 7.982627853901193


In [ ]:
for i in range(4,5):  
  test_image_dir = Path('/content/drive/MyDrive/Data/CNN ICA/source3/v{}'.format(i))
  test_filepaths = pd.Series(list(test_image_dir.glob(r'**/*.png')), name='Filepath').astype(str)
  test_bpms = pd.Series(test_filepaths.apply(lambda x: (os.path.split(os.path.split(x)[1])[1])[:-4]), name='BPMS').astype(np.float64)
  test_images = pd.concat([test_filepaths, test_bpms], axis=1).sample(frac=1.0, random_state=1).reset_index(drop=True)
  test_image_df = test_images.sample(len(test_images), random_state = 1).reset_index(drop=True)
  test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
  )
  test_images_flow = test_generator.flow_from_dataframe(
      dataframe=test_image_df,
      x_col='Filepath',
      y_col='BPMS',
      target_size=(216, 144),
      color_mode='rgb',
      class_mode='raw',
      batch_size=32,
      shuffle=False
  )
  predicted_bpm = np.squeeze(model.predict(test_images_flow))
  true_bpm  = test_images_flow.labels
  print("v" + str(i) + ": " + str(np.mean(np.abs(predicted_bpm-true_bpm))))

Found 105 validated image filenames.
v4: 7.943319598503688


In [ ]:
test_image_dir = Path('/content/drive/MyDrive/Data/CNN ICA/source3/v3-v5')
test_filepaths = pd.Series(list(test_image_dir.glob(r'**/*.png')), name='Filepath').astype(str)
test_bpms = pd.Series(test_filepaths.apply(lambda x: (os.path.split(os.path.split(x)[1])[1])[:-4]), name='BPMS').astype(np.float64)
test_images = pd.concat([test_filepaths, test_bpms], axis=1).sample(frac=1.0, random_state=1).reset_index(drop=True)
test_image_df = test_images.sample(len(test_images), random_state = 1).reset_index(drop=True)
test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
  )
test_images_flow = test_generator.flow_from_dataframe(
      dataframe=test_image_df,
      x_col='Filepath',
      y_col='BPMS',
      target_size=(216, 144),
      color_mode='rgb',
      class_mode='raw',
      batch_size=32,
      shuffle=False
  )
predicted_bpm = np.squeeze(model.predict(test_images_flow))
true_bpm  = test_images_flow.labels
print("v" + str(i) + ": " + str(np.mean(np.abs(predicted_bpm-true_bpm))))

Found 210 validated image filenames.
v4: 8.533449242115305
